In [1]:
import itertools
import math as m
import os 
import networkx as nx

In [2]:
def read_input(graphfile):
    
    trip_data = open(graphfile,'r').read().split('\n')
    i = 0
    listOfGraphs = {}
    k = 0
    
    while(True):
        if "#" in trip_data[i]:
            i = i+1
            N = int(trip_data[i])
            edges = list()
            while(True):
                i = i+1;
                if "#" in trip_data[i]:
                    break;
                if "" == trip_data[i]:
                    break;
                line = trip_data[i].split(" ")
                edges.append((line[0],line[1],line[2],line[3]))
                if i >= len(trip_data)-1:
                    break;
            G = {'Nodes':N,'list of edges': edges}
            listOfGraphs[k] = G
            k +=1 
        if i >= len(trip_data)-1:
                    break; 

    return listOfGraphs;

In [3]:
def flowMultipleDecomposition(data,K):
    
   
    # calculate the minimal flow decomposition based on such graph
    V = data['vertices']
    E = data['edges']
    f_low = data['flows_low']
    f_up = data['flows_up']
    W = 1e5
    S = {str(min(map(int,V)))}
    D = {str(max(map(int,V)))}
    
    # import library
    from docplex.mp.model import Model

    # model name
    Flow = Model(name = 'Flow Decomposition')

    # define variables
    x = {(i,j,k): Flow.binary_var(name="x_{0}_{1}_{2}".format(i,j,k)) for (i,j) in E for k in range(0,K)}
    w = {(k): Flow.integer_var(name="w_{0}_".format(k)) for k in range(0,K)}
    z = {(i,j,k): Flow.integer_var(name="z_{0}_{1}_{2}".format(i,j,k)) for (i,j) in E for k in range(0,K)}

    # flow conservation
    for k in range(0,K):
        for i in V:
            if i in S:
                Flow.add_constraint(sum(x[i,j,k] for j in V if i!=j and(i,j) in E) - sum(x[j,i,k] for j in V if i!=j and (j,i) in E) == 1)  
            if i in D:
                Flow.add_constraint(sum(x[i,j,k] for j in V if i!=j and (i,j) in E) - sum(x[j,i,k] for j in V if i!=j and (j,i) in E) == -1)
            if i not in S and i not in D:
                Flow.add_constraint(sum(x[i,j,k] for j in V if i!=j and (i,j) in E) - sum(x[j,i,k] for j in V if i!=j and (j,i) in E) == 0)
    
    # flow balance
    for (i,j) in E:
        Flow.add_constraint(f_up[i,j] >= sum(z[i,j,k] for k in range(0,K)))
        Flow.add_constraint(f_low[i,j] <= sum(z[i,j,k] for k in range(0,K)))
        
    
    # linearization
    for (i,j) in E:
        for k in range(0,K):
            Flow.add_constraint(z[i,j,k] <= W*x[i,j,k])
            Flow.add_constraint(w[k] - (1 - x[i,j,k])*W <= z[i,j,k])
            Flow.add_constraint(z[i,j,k] <= w[k])
    
    # objective function
    Flow.minimize(1)

    Flow.parameters.threads = 1
    Flow.parameters.mip.display = 0
    Flow.parameters.timelimit = 60
    #Flow.parameters.conflict.display = 1
    #Flow.parameters.emphasis.mip = 0
    #Flow.parameters.mip.pool.intensity = 4
    
    sol = Flow.solve(log_output=False)
    print('status',Flow.solve_details.status)

    if Flow.solve_details.status == 'integer infeasible' or Flow.solve_details.status == 'time limit exceeded, no integer solution':
        data['messsage'] = "infeasible"
        return data

    elif Flow.solve_details.status == 'time limit exceeded':
        print('time limit')
  
    print('FD time',Flow.solve_details.time)
               
    data['weights'] = K
    data['message'] = 'solved'
    data['runtime'] = Flow.solve_details.time
    
    return data;
    

In [4]:
def FD_Algorithm(data):
    
    listOfEdges = data['edges']
    solutionMap = data['graph']
    solutionSet = 0
    Kmin = data['minK']
    solutionWeights = 0

    for i in range(1,len(listOfEdges)+1):
        data = flowMultipleDecomposition(data,i)
        if data['message'] == "solved":
            solutionWeights = data['weights']
            break;
        
    
    return data

In [5]:
def SolveInstances(Graphs,outfile):
    
    fp = open(outfile,'w+')
    
    for i in range(0,1): 
        
        f_low = {}
        f_up = {}
        Edges = set()
        V = set()
        listOfEdges = Graphs[i]['list of edges']
        for k in range(0,len(listOfEdges)):
            (a,b,c,d) = (listOfEdges[k])
            Edges.add((a,b))
            V.add(a)
            V.add(b)
            f_low[a,b] = int(float(c))
            f_up[a,b] = int(float(d))
            
        
        # creation of graphs
        G = nx.DiGraph()
        G.add_edges_from(Edges,weights = f_low)
        G.add_nodes_from(V)
        
        
        # calculating topological order
        G_top = nx.topological_sort(G)
        G_top_list = list(nx.topological_sort(G))
        topological_cuts = sorted(G.degree, key=lambda x: x[1], reverse=True)[0][1]
        
        # definition of data
        
        data = {'edges' : Edges,
                'flows_up' : f_up,
                'flows_low' : f_low,
                'vertices' : V,
                'graph' : G,
                'Kmax' : len(Edges),
                'weights' : {},
                'message': {},
                'minK': topological_cuts,
                'runtime': 0,
        }
        
        data = FD_Algorithm(data)        
        
        print(data['weights'],data['runtime'],file=fp)
        
    
    return 0

In [6]:
path = "./Example/Inexact/"
text_files = [f for f in os.listdir(path) if f.endswith('.graph')]
outputfile = "results_cplex_inexact.txt"

for i in range(0,len(text_files)): 
    filename = text_files[i]
    data = read_input(''.join([path,filename]))
    data = SolveInstances(data,outputfile)
    
print("Done")

status integer infeasible
status integer infeasible
status integer optimal solution
FD time 0.009948015213012695
Done
